In [ ]:
input_html_path = ""
output_csv_path = ""

In [ ]:
!pip install beautifulsoup4
!pip install pandas
import re
from pandas import pd
from bs4 import BeautifulSoup

def clean_number(text):
    """Remove commas from numbers and convert to integer"""
    return int(text.replace(',', ''))

def extract_profiles(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    profiles = []
    columns = [
        'Onlyfans_name', 'Name', 'Subscribers', 'Likes', 'Photos', 'Videos', 'Price',
        'twitter_link','Onlyfans_link' 'instagram_link', 'tiktok_link', 'fansly_link', 'Info'
    ]
    
    # Find all result divs
    results = soup.find_all('div', class_='result')
    
    for result in results:
        profile = {}
        
        # Get Onlyfans_name from img title
        img = result.find('img')
        if img:
            name = img.get('title', '')
            profile['Onlyfans_name'] = name
            profile['Onlyfans_link'] = f'https://onlyfans.com/{name}'
            
        # Get Name from h3
        h3 = result.find('h3')
        if h3:
            profile['Name'] = h3.text.strip()
            
            
        # Get Info from about div
        about = result.find('div', class_='pb-3 text-muted about')
        if about:
            profile['Info'] = about.text.strip()
            
        # Get profile info (likes, photos, videos, price)
        profile_info = result.find('div', class_='profile-info')
        if profile_info:
            # Extract Likes

                
            # Extract Photos
            spans = profile_info.find_all('span', class_='mr-3')
            for span in spans:
                if 'Photos' in span.get_text():
                    profile['Photos'] = clean_number(span.get_text().replace('Photos', '').strip())
                if 'Videos' in span.get_text():
                    profile['Videos'] = clean_number(span.get_text().replace('Videos', '').strip())
                if 'Subscribers' in span.get_text():
                    profile['Subscribers'] = clean_number(span.get_text().replace('Subscribers', '').strip())
                if 'Likes' in span.get_text():
                    profile['Likes'] = clean_number(span.get_text().replace('Likes', '').strip())
                
            # Extract Price
            price_span = profile_info.find('span', class_='')
            if price_span:
                if price_span.find('svg', {'title': 'free trial link'}):
                    profile['Price'] = 'free trial link'
                elif price_span.find('strong'):
                    profile['Price'] = price_span.find('strong').text.strip()
                    
        # Get social media links
        social_div = result.find('div', class_='float-right profile-social')
        if social_div:
            social_links = social_div.find_all('a')
            for link in social_links:
                platform = link.get('title', '').lower()
                if platform:
                    profile[f'{platform}_link'] = link.get('href', '')
                    if f'{platform}_link' not in columns:
                        columns.append(f'{platform}_link')
                    
        profiles.append(profile)
        print(f"Profiles scraped:{len(profiles)}")
    print(columns)    
    return profiles

with open(input_html_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

profiles = extract_profiles(html_content) 
columns = ['Onlyfans_name', 'Name', 'Subscribers', 'Likes', 'Photos', 
           'Videos', 'Price','Onlyfans_link','instagram_link', 'twitter_link',  'tiktok_link',
           'fansly_link', 'pornhub_link','patreon_link', 'Info', ]


df = pd.DataFrame(profiles, columns=columns)
df = df.drop_duplicates(subset=['Onlyfans_name'], keep='first')
df.to_csv(output_csv_path, index=False)
df